# Confidence Intervals

Statistical Inference é o processo de analisar uma amostra de dados afim de obter insght (alguma informação) sobre a população na qual os dados foram removidos e investigar diferenças entre amostras. Em analise de dados queremos olhar pra um grande conjunto de dados, embora em alguns casos olhar para todo o conjunto seja inviável. Em alguns casos é suficiente olhar apenas para um subconjunto da população. 

# Point Estimates

Point Estimates são estimativas dos parâmetros da população baseado em uma amostra dos dados. Por exemplo, podemos estar interessados em saber qual a média da idade da população votante do E.U.A. Poderiamos tomar os dados da idade da população votante registrada e usar a média da idade como Point Estimate para dizer que é uma estimativa da idade média de toda a população. A média de uma amostra é conhecida como média amostral. 

A média amostral normalmente não é a mesma que a média da população. Esta diferença pode ocorrer por diversos fatores, como: biased sample (viés na amostra), aleatóriedade para obter uma amostra de uma população etc.. Vamos investigar um ponto estimado gerando uma população aleatória de idades e então obtendo uma amostra da população e estimando a média. 

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import random
import math

In [3]:
np.random.seed(10)
population_ages1 = stats.poisson.rvs(loc = 18, mu = 35, size = 150000)
population_ages2 = stats.poisson.rvs(loc = 18, mu = 10, size = 100000)
population_ages = np.concatenate((population_ages1, population_ages2))
population_ages.mean()

In [4]:
np.random.seed(6)
sample_ages = np.random.choice(a = population_ages, size = 500)
sample_ages.mean(), population_ages.mean() - sample_ages.mean()


Nosso ponto estimado obtido a partir de uma amostra de 500 individuos subestimou a verdadeira média da idade da população em 0.6 anos, mas esta próximo. Isto ilustra como podemos uma estimativa não tão distante de uma população com uma amostra relativamente pequena de individuos. 

Outro ponto estimado que pode ser interessante é a proporção da população pertence a uma categoria ou subgrupo. Por exemplo, podemos querer saber qual a etinia de cada vontante que investigamos anteriormente. Pode-se fazer uma estimativa desta proporção retirando uma amostra e depois verificando na população.

In [5]:
random.seed(10)
population_races = (["white"]*100000) + (["black"]*50000) +\
                   (["hispanic"]*50000) + (["asian"]*25000) +\
                   (["other"]*25000)
demo_sample = random.sample(population_races, 1000)

for race in set(demo_sample):
    print(race + ' proportion estimate:')
    print(demo_sample.count(race)/1000)

In [6]:
population_races.count('white')/250000 - demo_sample.count('white')/1000

Note que subestimamos a proporção de brancos na população por um fator de 0.021 que não é muito grande. 

# Sampling Distributions and The Central Limit Theorem

Muitas vezes é assumido que os dados seguem uma distribuição normal, devido as propriedades desta distribuição, como: simetria e que a maioria dos dados estarem a 1sigma da média. Contudo, nem sempre os dados seguem uma distribuição normal e são simetricos, vamos investigar a assimetria (skewed) dos dados apresentados anteriormente.




In [7]:
pd.DataFrame(population_ages).hist(bins = 58, 
                                  range=(17.5,75.5),
                                  figsize = (9,9))
stats.skew(population_ages)

Esta distribuição apresenta skewed pequenos, mas vemos pelo plot que claramente não é uma distribuição normal.

Amostras tiradas desta população devem ter aproximadamente a mesma forma e skew:

In [8]:
pd.DataFrame(sample_ages).hist(bins = 58, 
                                  range=(17.5,75.5),
                                  figsize = (9,9))
stats.skew(sample_ages)

O teorema do limite centra é um dos mais importantes teoremas da estatística. Este teorema airma que não importa o tipo de distribuição que nossa população tenha, se pegarmos um número grande de amostras a distribuição da média dessas amostras tendera a uma distribuição normal. E portanto, podemos trata a média amostral como sendo uma distribuição normal.

Para ilustrar essa ideia criaremos uma distribuição amostral tomando 200 amostras da nossa população e então faremos 200 estimativas da média:

In [9]:
np.random.seed(10)

point_estimates = []
for x in range(200):
    sample = np.random.choice(a = population_ages, size = 500)
    point_estimates.append(sample.mean())
pd.DataFrame(point_estimates).plot(kind = 'density',
                                  figsize = (9,9),
                                  xlim = (41.0, 45.0))
    

O plot acima ilustracomo como a distribuição amostral aproxima-se de uma distribuição normal. Além disso, a média da distribuição amostral aproxima-se da média verdadeira da população. 

In [10]:
np.array(point_estimates).mean(), population_ages.mean(),np.array(point_estimates).mean() - population_ages.mean() 


# Confidence Intervals



Um ponto estimado pode fornecer uma ideia aproximada de um parâmetro populacional, como exemplo, a média. Contudo, as estimativas são propensas a erros e tomar multiplas amostras para obter melhores estimativas nem sempre pode ser viável. 

O intervalo de confiança (C.I) é um intervalo de valores acima e abaixo da estimativa que captura o parâmetro populacional verdadeiro em um dado nível de confiança. Exemplo, se você quer ter 95% de chance de capturar o parâmetro verdadeiro com uma estimativa, você deve fiar o nível de confianã em 95%. 
Grandes níveis de confiança (C.L) implicam em intervalos de confiança maiores. 

Calculamos o intervalo de confiança tomando o ponto estimado e somando e subtraindo uma margem de erro. A margem de erro depende do C.L, do tipo de distribuição dos dados e do tamanho da amostra. A forma que calculamos a margem de erro depende de sabermos ou não o desvio padrão amostral. 

Se conhecemos o desvio padrão populacional, a margem de erro é dada por 

z*sigma/sqrt(n), onde:

sigma é o desvio padrão populacional

sqrt(n) é a raiz quadrado do tamanho da amostra

z é um nº conhecido como valor crítico.

O valor crítico é o nº de desvio padrão que você deveria andar (partindo da média da distribuição normal) para capturar a proporção associada com o C.L desejando. Por exemplo, sabemos que, em uma distribuição normal, aproximadamente 95% dos dados encontram-se a 2sigma da média, portanto, neste caso, z-crticial = 2, para C.L de 95%. Podemos conseguir o valor exato de z-crtical usando a função *stats.norm.ppf()*

Vamos calcular um intervalo de confiança de 95% para a nossa média estimada:

In [11]:
np.random.seed(10)
sample_size = 1000
sample = np.random.choice(a = population_ages, size = sample_size)
sample_mean = sample.mean()

z_critical = stats.norm.ppf(q = 0.975)
z_critical
print("z-critical value:"),
print(z_critical)
pop_stdev = population_ages.std()

margin_of_error = z_critical*(pop_stdev/math.sqrt(sample_size))


confidence_interval = (sample_mean - margin_of_error, sample_mean + margin_of_error)
confidence_interval
print("confidence interval:")
print(confidence_interval)


*Note que usamos q = 0.975 pois a distribuição tem duas caldas e é simétrica* 

Veja que o intervalo compreende o valor médio populacional. 

Vamos criar vários intervalos de confiança e plotalos, para entender melhor o que signica capturar a média verdadeira:

In [12]:
np.random.seed(12)

sample_size = 1000

intervals = []
sample_means = []

for sample in range(25):
    sample = np.random.choice(a = population_ages, size = sample_size)
    sample_mean = sample.mean()
    sample_means.append(sample_mean)

    
    z_critical = stats.norm.ppf(q = 0.975)

    pop_stdev = population_ages.std()

    margin_of_error = z_critical* (pop_stdev/math.sqrt(sample_size))
    confidence_interval = (sample_mean - margin_of_error, sample_mean + margin_of_error)
    intervals.append(confidence_interval)
    


In [13]:
plt.plot(figsize = (9,9))

plt.errorbar(x = np.arange(0.1,25,1),
            y = sample_means,
            yerr= [(top - bot)/2 for bot,top in intervals],
            fmt = 'o')

plt.hlines(xmin = 0, xmax = 25,
          y = population_ages.mean(),
          linewidth = 2.0,
          color = 'red')

Note que no gráfico acima apenas uma das barras não intercepta a barra vermelha representado o valor médio verdadeiro. Contudo, isto é justamente o que significa um nivel de confiança de 95%, pois um nível de confiança de 95% cria intervalos de confiança que capturam o valor verdadeiro 95% das vezes.  

Se não conhecemos o desvio padrão populacional precisamos usar o desvio padrão amostral para construir o C.I. Contudo, isto acarreta em erros maiores. Para levar em conta este erro, em vez de usarmos o z-critical value passamos a usar o t-critical value. O z-critical é obtido a partir da função stats.norm.ppf() pois ele vem da distribuição normal, o t-critical é obtido de uma distribuição conhecida como distribuição-t. Uma das caracteristicas da distribuição-t na qual quando tomamos um limite amostral muito grande ela se parece com a distribuição normal. O scipy name para a distribuição-t é simplesmente t, assim usamos, stats.t.ppf() para obter o t-critical. 

Para exemplificar, vamos tomar uma amostra menor e construiremos o intervalo de confiança sem usar o desvio padrão populacional.

In [14]:
np.random.seed(10)

sample_size = 25

sample = np.random.choice(a = population_ages, size = sample_size)
sample_mean = sample.mean()



t_critical = stats.t.ppf(q = 0.975, df = 24)

sample_stdev = sample.std(ddof =1)

sigma = sample_stdev/math.sqrt(sample_size)

margin_of_error = t_critical*sigma

confidence_interval = (sample_mean - margin_of_error, sample_mean + margin_of_error)

print("t-critical value:",  t_critical)
print("Confidence interval:" , confidence_interval)


*Note que o t-critical value é maior que o z-critical*. Além disso, usamos df = 24, pois nestes casos o degree of freedon é o tamanho da amostra menos um. 

Em vez de calcular manualmente um intervalo de confiança usando a média estimada, podemos fazer isso diretamente pela função stats.t.interval():

In [15]:
np.random.seed(10)

sample_size = 25
sample = np.random.choice(a = population_ages, size = sample_size)
sample_mean = sample.mean()
sigma = sample.std(ddof = 1)/math.sqrt(sample_size)

stats.t.interval(alpha = 0.95, #C.L
                df = 24,
                loc = sample_mean,
                scale = sigma)

Podemos calcular um intervalo de confiança para uma estimativa da proporção populacional. Neste caso, a margem de erro é dada por:

z*sqrt(p(1-p)/n)

z: z-critical

p: é a proporção estimada

n: tamanho amostral.

Lembre que para Hispanicos encontramos uma proporção de 0.192, vamos calcular um intervalo de confiança de C.L = 95%:


In [16]:
z_critical = stats.norm.ppf(q = 0.975)
p = 0.192
n = 1000
margin_of_error = z_critical* math.sqrt(p*(1-p)/n)
confidence_interval = ( p - margin_of_error, p +  margin_of_error)

confidence_interval

De forma analoga podemos usar *stats.distribuition.interval()* para calcular os intervalos diretamente:

In [19]:
confidence_interval = stats.norm.interval(alpha = 0.95,
                                         loc = 0.192,
                                         scale = math.sqrt(p*(1-p)/n))
confidence_interval